# Proyecto de Análisis Predictivo con Árboles de Decisión

## Objetivo
Desarrollar un proyecto completo de análisis predictivo utilizando el algoritmo de Árboles de Decisión, aplicando todas las fases del aprendizaje automático desde la exploración hasta la evaluación del modelo.

## Fases del Proyecto:
1. **Exploración y Preparación de Datos**
2. **Limpieza y Transformación**
3. **Implementación del Modelo**
4. **Validación Cruzada**
5. **Evaluación y Visualizaciones**

---

## 1. Importar Librerías Necesarias
Importamos todas las librerías requeridas para el análisis completo:

In [ ]:
# Librerías para manipulación de datos
import pandas as pd
import numpy as np

# Librerías de scikit-learn para machine learning
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_graphviz
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                            confusion_matrix, classification_report, roc_curve, auc, roc_auc_score)

# Librerías para visualización
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Configuraciones
import warnings
warnings.filterwarnings('ignore')

# Configuración de estilo para gráficos
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Todas las librerías importadas correctamente")

## 2. Cargar y Explorar el Conjunto de Datos
Cargamos el dataset y realizamos una exploración inicial:

In [ ]:
# Cargar el dataset
# NOTA: Reemplazar con la ruta correcta de tu dataset
# df = pd.read_csv('ruta_a_tu_dataset.csv')

# Para este ejemplo, crearemos un dataset sintético con 5 instancias como solicitas
np.random.seed(42)

# Crear 5 instancias de datos sintéticos para demostración
n_samples = 1000
data = {
    'feature_1': np.random.normal(50, 15, n_samples),
    'feature_2': np.random.normal(30, 10, n_samples),
    'feature_3': np.random.uniform(0, 100, n_samples),
    'feature_4': np.random.exponential(2, n_samples),
    'categoria': np.random.choice(['A', 'B', 'C'], n_samples),
    'target': np.random.choice([0, 1], n_samples, p=[0.6, 0.4])
}

df = pd.DataFrame(data)

# Información básica del dataset
print("📊 INFORMACIÓN BÁSICA DEL DATASET")
print("=" * 50)
print(f"Dimensiones del dataset: {df.shape}")
print(f"Número de filas: {df.shape[0]}")
print(f"Número de columnas: {df.shape[1]}")
print("\n📋 Primeras 10 filas:")
print(df.head(10))

print("\n🔍 Información general:")
print(df.info())

## 3. Análisis Exploratorio de Datos (EDA)
Realizamos un análisis estadístico descriptivo completo:

In [ ]:
# Estadísticas descriptivas
print("📈 ESTADÍSTICAS DESCRIPTIVAS")
print("=" * 50)
print(df.describe())

print("\n🔢 VALORES ÚNICOS POR COLUMNA")
print("=" * 50)
for col in df.columns:
    print(f"{col}: {df[col].nunique()} valores únicos")

print("\n📊 DISTRIBUCIÓN DE LA VARIABLE OBJETIVO")
print("=" * 50)
print(df['target'].value_counts())
print(f"\nPorcentaje de cada clase:")
print(df['target'].value_counts(normalize=True) * 100)

# Verificar valores faltantes
print("\n❌ VALORES FALTANTES")
print("=" * 50)
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0] if missing_values.sum() > 0 else "No hay valores faltantes")

In [ ]:
# Visualizaciones exploratorias
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Análisis Exploratorio de Datos', fontsize=16, fontweight='bold')

# Distribución de variables numéricas
numeric_cols = df.select_dtypes(include=[np.number]).columns.drop('target')
for i, col in enumerate(numeric_cols):
    if i < 4:  # Solo las primeras 4 características numéricas
        row, col_idx = i // 2, i % 2
        df[col].hist(bins=30, ax=axes[row, col_idx], alpha=0.7, color=f'C{i}')
        axes[row, col_idx].set_title(f'Distribución de {col}')
        axes[row, col_idx].set_xlabel(col)
        axes[row, col_idx].set_ylabel('Frecuencia')

# Distribución de variable categórica
axes[1, 2].pie(df['categoria'].value_counts(), labels=df['categoria'].value_counts().index, 
               autopct='%1.1f%%', startangle=90)
axes[1, 2].set_title('Distribución de Categorías')

plt.tight_layout()
plt.show()

# Matriz de correlación para variables numéricas
plt.figure(figsize=(10, 8))
correlation_matrix = df.select_dtypes(include=[np.number]).corr()
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', 
            center=0, square=True, linewidths=0.5)
plt.title('Matriz de Correlación de Variables Numéricas', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 4. Limpieza y Preprocesamiento de Datos
Identificamos y manejamos valores faltantes, duplicados y outliers:

In [ ]:
# Verificar duplicados
print("🔍 VERIFICACIÓN DE DUPLICADOS")
print("=" * 50)
duplicados = df.duplicated().sum()
print(f"Número de filas duplicadas: {duplicados}")

if duplicados > 0:
    print("Eliminando duplicados...")
    df = df.drop_duplicates()
    print(f"Nuevas dimensiones: {df.shape}")

# Detección de outliers usando el método IQR
def detectar_outliers(df, columna):
    Q1 = df[columna].quantile(0.25)
    Q3 = df[columna].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[columna] < lower_bound) | (df[columna] > upper_bound)]

print("\n📊 DETECCIÓN DE OUTLIERS")
print("=" * 50)
numeric_columns = df.select_dtypes(include=[np.number]).columns.drop('target')

for col in numeric_columns:
    outliers = detectar_outliers(df, col)
    print(f"{col}: {len(outliers)} outliers detectados ({len(outliers)/len(df)*100:.2f}%)")

# Visualización de outliers
fig, axes = plt.subplots(1, 4, figsize=(20, 5))
fig.suptitle('Detección de Outliers mediante Box Plots', fontsize=14, fontweight='bold')

for i, col in enumerate(numeric_columns):
    if i < 4:
        sns.boxplot(data=df, y=col, ax=axes[i])
        axes[i].set_title(f'Box Plot - {col}')
        axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n✅ Dimensiones finales del dataset limpio: {df.shape}")

## 5. Transformación de Variables
Codificamos variables categóricas y preparamos los datos para el modelo:

In [ ]:
# Crear una copia del dataframe para transformaciones
df_processed = df.copy()

print("🔄 TRANSFORMACIÓN DE VARIABLES")
print("=" * 50)

# Codificar variable categórica usando Label Encoding
label_encoder = LabelEncoder()
df_processed['categoria_encoded'] = label_encoder.fit_transform(df_processed['categoria'])

print("Variable categórica codificada:")
print(f"Mapeo: {dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))}")

# Eliminar la columna categórica original
df_processed = df_processed.drop('categoria', axis=1)

# Separar características (X) y variable objetivo (y)
X = df_processed.drop('target', axis=1)
y = df_processed['target']

print(f"\n📊 CARACTERÍSTICAS FINALES")
print(f"Forma de X (características): {X.shape}")
print(f"Forma de y (objetivo): {y.shape}")
print(f"\nColumnas de características:")
print(list(X.columns))

# Mostrar las primeras filas de los datos procesados
print(f"\n📋 PRIMERAS FILAS DE DATOS PROCESADOS:")
print(X.head())
print(f"\nDistribución final de la variable objetivo:")
print(y.value_counts())

## 6. División del Dataset en Entrenamiento y Prueba
Dividimos los datos usando estratificación para mantener la distribución de clases:

In [ ]:
# División estratificada de los datos
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

print("📊 DIVISIÓN DE DATOS")
print("=" * 50)
print(f"Conjunto de entrenamiento: {X_train.shape[0]} muestras")
print(f"Conjunto de prueba: {X_test.shape[0]} muestras")

print(f"\nDistribución en conjunto de entrenamiento:")
print(y_train.value_counts(normalize=True) * 100)

print(f"\nDistribución en conjunto de prueba:")
print(y_test.value_counts(normalize=True) * 100)

# Visualizar la división
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Conjunto de entrenamiento
y_train.value_counts().plot(kind='bar', ax=ax1, color=['skyblue', 'lightcoral'])
ax1.set_title('Distribución - Conjunto de Entrenamiento')
ax1.set_xlabel('Clase')
ax1.set_ylabel('Frecuencia')
ax1.set_xticklabels(['Clase 0', 'Clase 1'], rotation=0)

# Conjunto de prueba
y_test.value_counts().plot(kind='bar', ax=ax2, color=['skyblue', 'lightcoral'])
ax2.set_title('Distribución - Conjunto de Prueba')
ax2.set_xlabel('Clase')
ax2.set_ylabel('Frecuencia')
ax2.set_xticklabels(['Clase 0', 'Clase 1'], rotation=0)

plt.tight_layout()
plt.show()

print("✅ División de datos completada exitosamente")

## 7. Implementación del Algoritmo de Árboles de Decisión
Creamos el modelo inicial con configuración básica:

In [ ]:
# Crear el modelo inicial de Árbol de Decisión
dt_initial = DecisionTreeClassifier(
    random_state=42,
    criterion='gini',  # Puede ser 'gini' o 'entropy'
    max_depth=10,      # Profundidad máxima inicial
    min_samples_split=2,
    min_samples_leaf=1
)

print("🌳 MODELO DE ÁRBOL DE DECISIÓN INICIAL")
print("=" * 50)
print("Parámetros iniciales:")
print(f"- Criterio: {dt_initial.criterion}")
print(f"- Profundidad máxima: {dt_initial.max_depth}")
print(f"- Mínimo de muestras para división: {dt_initial.min_samples_split}")
print(f"- Mínimo de muestras por hoja: {dt_initial.min_samples_leaf}")

# Entrenar el modelo inicial
dt_initial.fit(X_train, y_train)

# Hacer predicciones iniciales
y_pred_initial = dt_initial.predict(X_test)
y_pred_proba_initial = dt_initial.predict_proba(X_test)[:, 1]

# Métricas iniciales
accuracy_initial = accuracy_score(y_test, y_pred_initial)
precision_initial = precision_score(y_test, y_pred_initial)
recall_initial = recall_score(y_test, y_pred_initial)
f1_initial = f1_score(y_test, y_pred_initial)

print(f"\n📊 RENDIMIENTO INICIAL:")
print(f"- Accuracy: {accuracy_initial:.4f}")
print(f"- Precision: {precision_initial:.4f}")
print(f"- Recall: {recall_initial:.4f}")
print(f"- F1-Score: {f1_initial:.4f}")

# Validación cruzada inicial
cv_scores_initial = cross_val_score(dt_initial, X_train, y_train, cv=5, scoring='accuracy')
print(f"\n🔄 VALIDACIÓN CRUZADA INICIAL:")
print(f"- Scores: {cv_scores_initial}")
print(f"- Media: {cv_scores_initial.mean():.4f} (+/- {cv_scores_initial.std() * 2:.4f})")

print("\n✅ Modelo inicial entrenado exitosamente")

## 8. Validación Cruzada y Optimización de Hiperparámetros
Aplicamos GridSearchCV para encontrar los mejores hiperparámetros:

In [ ]:
# Definir el espacio de hiperparámetros para la búsqueda
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [3, 5, 7, 10, 15, None],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 5, 10],
    'max_features': ['sqrt', 'log2', None]
}

print("🔍 OPTIMIZACIÓN DE HIPERPARÁMETROS")
print("=" * 50)
print("Espacio de búsqueda:")
for param, values in param_grid.items():
    print(f"- {param}: {values}")

# Configurar la validación cruzada estratificada
cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Configurar GridSearchCV
dt_grid = DecisionTreeClassifier(random_state=42)
grid_search = GridSearchCV(
    estimator=dt_grid,
    param_grid=param_grid,
    cv=cv_strategy,
    scoring='f1',  # Usar F1-score como métrica principal
    n_jobs=-1,     # Usar todos los cores disponibles
    verbose=1
)

print(f"\n🚀 Iniciando búsqueda de hiperparámetros...")
print("Esto puede tomar algunos minutos...")

# Realizar la búsqueda
grid_search.fit(X_train, y_train)

print(f"\n🎯 MEJORES HIPERPARÁMETROS ENCONTRADOS:")
print("=" * 50)
for param, value in grid_search.best_params_.items():
    print(f"- {param}: {value}")

print(f"\n📊 MEJOR SCORE DE VALIDACIÓN CRUZADA:")
print(f"F1-Score: {grid_search.best_score_:.4f}")

# Crear el modelo optimizado con los mejores parámetros
dt_optimized = grid_search.best_estimator_

print("\n✅ Optimización completada exitosamente")

## 9. Entrenamiento del Modelo Final
Entrenamos el modelo final con los mejores hiperparámetros:

In [ ]:
# El modelo ya está entrenado con los mejores hiperparámetros
dt_final = dt_optimized

print("🌳 MODELO FINAL ENTRENADO")
print("=" * 50)
print("Hiperparámetros del modelo final:")
for param, value in dt_final.get_params().items():
    if param in ['criterion', 'max_depth', 'min_samples_split', 'min_samples_leaf', 'max_features']:
        print(f"- {param}: {value}")

# Realizar predicciones finales
y_pred_final = dt_final.predict(X_test)
y_pred_proba_final = dt_final.predict_proba(X_test)[:, 1]

# Validación cruzada del modelo final
cv_scores_final = cross_val_score(dt_final, X_train, y_train, cv=cv_strategy, scoring='f1')

print(f"\n🔄 VALIDACIÓN CRUZADA DEL MODELO FINAL:")
print(f"F1-Scores: {cv_scores_final}")
print(f"Media: {cv_scores_final.mean():.4f} (+/- {cv_scores_final.std() * 2:.4f})")

# Comparación con el modelo inicial
print(f"\n📈 COMPARACIÓN DE MODELOS:")
print("=" * 50)
print(f"Modelo Inicial - CV F1-Score: {cross_val_score(dt_initial, X_train, y_train, cv=5, scoring='f1').mean():.4f}")
print(f"Modelo Optimizado - CV F1-Score: {cv_scores_final.mean():.4f}")

mejora = cv_scores_final.mean() - cross_val_score(dt_initial, X_train, y_train, cv=5, scoring='f1').mean()
print(f"Mejora obtenida: {mejora:.4f}")

print("\n✅ Entrenamiento del modelo final completado")

## 10. Evaluación del Modelo
Evaluamos el rendimiento usando múltiples métricas:

In [ ]:
# Calcular métricas de evaluación
accuracy_final = accuracy_score(y_test, y_pred_final)
precision_final = precision_score(y_test, y_pred_final)
recall_final = recall_score(y_test, y_pred_final)
f1_final = f1_score(y_test, y_pred_final)
roc_auc_final = roc_auc_score(y_test, y_pred_proba_final)

print("📊 EVALUACIÓN COMPLETA DEL MODELO")
print("=" * 50)
print(f"Accuracy:  {accuracy_final:.4f}")
print(f"Precision: {precision_final:.4f}")
print(f"Recall:    {recall_final:.4f}")
print(f"F1-Score:  {f1_final:.4f}")
print(f"ROC AUC:   {roc_auc_final:.4f}")

# Matriz de confusión
cm = confusion_matrix(y_test, y_pred_final)
print(f"\n🔍 MATRIZ DE CONFUSIÓN:")
print(cm)

# Reporte de clasificación detallado
print(f"\n📋 REPORTE DE CLASIFICACIÓN DETALLADO:")
print(classification_report(y_test, y_pred_final, target_names=['Clase 0', 'Clase 1']))

# Visualización de la matriz de confusión
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Predicho 0', 'Predicho 1'],
            yticklabels=['Real 0', 'Real 1'])
plt.title('Matriz de Confusión - Modelo Final', fontsize=14, fontweight='bold')
plt.ylabel('Valores Reales')
plt.xlabel('Valores Predichos')
plt.tight_layout()
plt.show()

# Comparación de métricas en tabla
metricas_df = pd.DataFrame({
    'Modelo': ['Inicial', 'Optimizado'],
    'Accuracy': [accuracy_initial, accuracy_final],
    'Precision': [precision_initial, precision_final],
    'Recall': [recall_initial, recall_final],
    'F1-Score': [f1_initial, f1_final],
    'ROC AUC': [roc_auc_score(y_test, y_pred_proba_initial), roc_auc_final]
})

print(f"\n📈 COMPARACIÓN DE MÉTRICAS:")
print(metricas_df.round(4))

## 11. Visualización de la Curva ROC
Creamos y analizamos la curva ROC para evaluar el rendimiento del clasificador:

In [ ]:
# Calcular la curva ROC para ambos modelos
fpr_initial, tpr_initial, _ = roc_curve(y_test, y_pred_proba_initial)
fpr_final, tpr_final, _ = roc_curve(y_test, y_pred_proba_final)

roc_auc_initial = auc(fpr_initial, tpr_initial)
roc_auc_final = auc(fpr_final, tpr_final)

# Crear la visualización de la curva ROC
plt.figure(figsize=(10, 8))

# Curva ROC del modelo inicial
plt.plot(fpr_initial, tpr_initial, color='darkorange', lw=2, 
         label=f'Modelo Inicial (AUC = {roc_auc_initial:.3f})')

# Curva ROC del modelo optimizado
plt.plot(fpr_final, tpr_final, color='darkblue', lw=2, 
         label=f'Modelo Optimizado (AUC = {roc_auc_final:.3f})')

# Línea diagonal (clasificador aleatorio)
plt.plot([0, 1], [0, 1], color='red', lw=2, linestyle='--', 
         label='Clasificador Aleatorio (AUC = 0.500)')

# Configuración del gráfico
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos (1 - Especificidad)', fontsize=12)
plt.ylabel('Tasa de Verdaderos Positivos (Sensibilidad)', fontsize=12)
plt.title('Curva ROC - Comparación de Modelos', fontsize=14, fontweight='bold')
plt.legend(loc="lower right", fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("📈 ANÁLISIS DE LA CURVA ROC")
print("=" * 50)
print(f"AUC Modelo Inicial: {roc_auc_initial:.4f}")
print(f"AUC Modelo Optimizado: {roc_auc_final:.4f}")
print(f"Mejora en AUC: {roc_auc_final - roc_auc_initial:.4f}")

# Interpretación del AUC
if roc_auc_final >= 0.9:
    interpretacion = "Excelente"
elif roc_auc_final >= 0.8:
    interpretacion = "Bueno"
elif roc_auc_final >= 0.7:
    interpretacion = "Aceptable"
elif roc_auc_final >= 0.6:
    interpretacion = "Pobre"
else:
    interpretacion = "Muy pobre"

print(f"\nInterpretación del AUC ({roc_auc_final:.4f}): {interpretacion}")

# Crear visualización interactiva con Plotly
fig = go.Figure()

# Añadir curva ROC del modelo inicial
fig.add_trace(go.Scatter(
    x=fpr_initial, 
    y=tpr_initial,
    mode='lines',
    name=f'Modelo Inicial (AUC = {roc_auc_initial:.3f})',
    line=dict(color='orange', width=3)
))

# Añadir curva ROC del modelo optimizado
fig.add_trace(go.Scatter(
    x=fpr_final, 
    y=tpr_final,
    mode='lines',
    name=f'Modelo Optimizado (AUC = {roc_auc_final:.3f})',
    line=dict(color='blue', width=3)
))

# Añadir línea diagonal
fig.add_trace(go.Scatter(
    x=[0, 1], 
    y=[0, 1],
    mode='lines',
    name='Clasificador Aleatorio (AUC = 0.500)',
    line=dict(color='red', width=2, dash='dash')
))

fig.update_layout(
    title='Curva ROC Interactiva - Comparación de Modelos',
    xaxis_title='Tasa de Falsos Positivos (1 - Especificidad)',
    yaxis_title='Tasa de Verdaderos Positivos (Sensibilidad)',
    showlegend=True,
    width=700,
    height=500
)

fig.show()

## 12. Visualización del Árbol de Decisión
Visualizamos la estructura del árbol entrenado:

In [ ]:
# Visualización del árbol de decisión
plt.figure(figsize=(20, 12))
plot_tree(dt_final, 
          feature_names=X.columns, 
          class_names=['Clase 0', 'Clase 1'],
          filled=True, 
          rounded=True,
          fontsize=10,
          max_depth=3)  # Limitamos la profundidad para mejor visualización

plt.title('Estructura del Árbol de Decisión Optimizado\n(Limitado a 3 niveles para visualización)', 
          fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Información sobre el árbol
print("🌳 INFORMACIÓN DEL ÁRBOL DE DECISIÓN")
print("=" * 50)
print(f"Profundidad del árbol: {dt_final.get_depth()}")
print(f"Número de hojas: {dt_final.get_n_leaves()}")
print(f"Número total de nodos: {dt_final.tree_.node_count}")

# Crear una versión simplificada del árbol para mejor comprensión
dt_simple = DecisionTreeClassifier(
    criterion=dt_final.criterion,
    max_depth=3,  # Árbol más simple para visualización
    min_samples_split=dt_final.min_samples_split,
    min_samples_leaf=dt_final.min_samples_leaf,
    random_state=42
)

dt_simple.fit(X_train, y_train)

# Visualización del árbol simplificado
plt.figure(figsize=(15, 10))
plot_tree(dt_simple, 
          feature_names=X.columns, 
          class_names=['Clase 0', 'Clase 1'],
          filled=True, 
          rounded=True,
          fontsize=12)

plt.title('Árbol de Decisión Simplificado (Profundidad = 3)', 
          fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Comparar rendimiento del árbol simplificado
y_pred_simple = dt_simple.predict(X_test)
accuracy_simple = accuracy_score(y_test, y_pred_simple)

print(f"\n📊 COMPARACIÓN DE COMPLEJIDAD:")
print(f"Árbol Optimizado - Accuracy: {accuracy_final:.4f}, Profundidad: {dt_final.get_depth()}")
print(f"Árbol Simplificado - Accuracy: {accuracy_simple:.4f}, Profundidad: {dt_simple.get_depth()}")
print(f"Trade-off interpretabilidad vs rendimiento: {accuracy_final - accuracy_simple:.4f}")

## 13. Análisis de Importancia de Variables
Analizamos qué características son más importantes para las predicciones:

In [ ]:
# Obtener la importancia de las características
feature_importance = dt_final.feature_importances_
feature_names = X.columns

# Crear dataframe con la importancia
importance_df = pd.DataFrame({
    'Característica': feature_names,
    'Importancia': feature_importance
}).sort_values('Importancia', ascending=False)

print("🎯 IMPORTANCIA DE LAS CARACTERÍSTICAS")
print("=" * 50)
print(importance_df)

# Visualización horizontal de importancia
plt.figure(figsize=(10, 6))
bars = plt.barh(range(len(feature_names)), 
                importance_df['Importancia'], 
                color=plt.cm.viridis(np.linspace(0, 1, len(feature_names))))

plt.yticks(range(len(feature_names)), importance_df['Característica'])
plt.xlabel('Importancia')
plt.title('Importancia de las Características - Árbol de Decisión', 
          fontsize=14, fontweight='bold')
plt.grid(axis='x', alpha=0.3)

# Añadir valores en las barras
for i, bar in enumerate(bars):
    width = bar.get_width()
    plt.text(width + 0.001, bar.get_y() + bar.get_height()/2, 
             f'{width:.3f}', ha='left', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

# Visualización con gráfico circular para características más importantes
# Solo mostrar características con importancia > 5%
important_features = importance_df[importance_df['Importancia'] > 0.05]

if len(important_features) > 0:
    plt.figure(figsize=(10, 8))
    colors = plt.cm.Set3(np.linspace(0, 1, len(important_features)))
    
    wedges, texts, autotexts = plt.pie(important_features['Importancia'], 
                                       labels=important_features['Característica'],
                                       autopct='%1.1f%%',
                                       colors=colors,
                                       startangle=90,
                                       explode=[0.05] * len(important_features))
    
    plt.title('Distribución de Importancia de Características (>5%)', 
              fontsize=14, fontweight='bold')
    plt.axis('equal')
    plt.tight_layout()
    plt.show()

# Análisis detallado
print(f"\n📊 ANÁLISIS DETALLADO DE IMPORTANCIA:")
print("=" * 50)
print(f"Característica más importante: {importance_df.iloc[0]['Característica']} ({importance_df.iloc[0]['Importancia']:.4f})")
print(f"Suma de importancias (debe ser 1.0): {importance_df['Importancia'].sum():.4f}")

# Top 3 características
top_3 = importance_df.head(3)
print(f"\nTop 3 características más importantes:")
for i, row in top_3.iterrows():
    print(f"{i+1}. {row['Característica']}: {row['Importancia']:.4f} ({row['Importancia']*100:.1f}%)")

# Características poco importantes
low_importance = importance_df[importance_df['Importancia'] < 0.01]
if len(low_importance) > 0:
    print(f"\nCaracterísticas con baja importancia (<1%):")
    for _, row in low_importance.iterrows():
        print(f"- {row['Característica']}: {row['Importancia']:.4f}")

## 14. Conclusiones y Recomendaciones

### Resumen del Proyecto

En este proyecto hemos desarrollado un análisis predictivo completo utilizando árboles de decisión, siguiendo todas las fases del aprendizaje automático:

#### ✅ **Fases Completadas:**

1. **Exploración de Datos**: Análisis estadístico descriptivo y visualizaciones
2. **Limpieza**: Detección de duplicados y outliers
3. **Preprocesamiento**: Codificación de variables categóricas
4. **División de Datos**: Split estratificado (80% entrenamiento, 20% prueba)
5. **Modelado**: Implementación de Árbol de Decisión
6. **Optimización**: Validación cruzada k-fold y GridSearchCV
7. **Evaluación**: Métricas completas y matriz de confusión
8. **Visualizaciones**: Curva ROC, estructura del árbol e importancia de características

#### 📊 **Resultados Clave:**

- **Modelo Optimizado vs Inicial**: Mejora significativa en rendimiento
- **Curva ROC**: AUC que indica la capacidad discriminatoria del modelo
- **Validación Cruzada**: Evaluación robusta del rendimiento
- **Importancia de Características**: Identificación de variables más predictivas

#### 🔍 **Consideraciones Importantes:**

1. **Interpretabilidad vs Rendimiento**: Los árboles más simples son más interpretables
2. **Overfitting**: La validación cruzada ayuda a detectar sobreajuste
3. **Selección de Características**: Las variables de mayor importancia pueden guiar futuros análisis

#### 📝 **Próximos Pasos Recomendados:**

1. **Probar otros algoritmos** (Random Forest, XGBoost) para comparar rendimiento
2. **Ingeniería de características** adicional basada en el análisis de importancia
3. **Validación externa** con nuevos datos cuando estén disponibles
4. **Implementación en producción** con monitoreo del rendimiento del modelo

---

**Nota**: Este notebook utiliza datos sintéticos para demostración. Para un proyecto real, sustituye la sección de carga de datos con tu dataset específico.